# 1. Data Portal Preprocessing

In [1]:
import pandas as pd

In [3]:
from scispacy.linking import EntityLinker
import spacy
import scispacy

In [5]:
from tqdm.auto import tqdm

In [8]:
path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/1_decoded"

In [3]:
nlp = spacy.load("en_core_sci_lg")

In [ ]:
linker_umls = EntityLinker(name="umls", threshold=0.85)

In [ ]:
#linker_rxnorm = EntityLinker(resolve_abbreviations=True, name="rxnorm")

In [ ]:
nlp.add_pipe(linker_umls)

In [ ]:
import json
with open(f"{path}/drug_names.json") as f:
    drug_names = json.load(f)

In [ ]:
from tqdm.auto import tqdm

In [178]:
def extract_umls_cuis(drug_name, nlp, linker):
    rows=[]
    doc=nlp(drug_name)
    entities = doc.ents
    if len(entities)>0: 
        #entity = entities[0]._.kb_ents ### loop through all and do the filter for types later -> get all substances linked to name!
        for entity in entities:
            if len(entity)>0:
                for ent in entity._.kb_ents:
                    link = linker_umls.kb.cui_to_entity[ent[0]]
                    rows.append({"drug_index": index, "drug_name": drug_name, "CUI":link.concept_id, "TUI":link.types, "name": link.canonical_name})
    return rows

from joblib import Parallel, delayed
rows =  Parallel(n_jobs=1)(delayed(extract_umls_cuis)(drug_name, nlp, linker_umls) for drug_name in tqdm(drug_names))
                    
drugs_umls = pd.DataFrame({"drug_index": [],"drug_name": [], "CUI":[], "TUI": [],"name": []}).append([item for sublist in rows for item in sublist], ignore_index=True)
drugs_umls

/data/analysis/ag-reils/ag-reils-shared/deps/miniconda3/envs/pl1.x/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/data/analysis/ag-reils/ag-reils-shared/deps/miniconda3/envs/pl1.x/lib/python3.7/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


,drug_index,drug_name,CUI,TUI,name
0,999.0,Simvastatin 40mg tablets,C0074554,"[T109, T121]",simvastatin
1,999.0,Simvastatin 40mg tablets,C0039225,[T122],Tablet Dosage Form
2,999.0,Simvastatin 40mg tablets,C0993159,[T122],Oral Tablet
3,999.0,Bendroflumethiazide 2.5mg tablets,C0992038,[T200],Bendroflumethiazide 2.5 MG
4,999.0,Bendroflumethiazide 2.5mg tablets,C0004975,"[T109, T121]",bendroflumethiazide
...,...,...,...,...,...
261724,999.0,"zinc oxide with icthammol, salicylic acid and ...",C0036079,"[T109, T121]",salicylic acid
261725,999.0,"zinc oxide with icthammol, salicylic acid and ...",C0073983,"[T109, T121]",salsalate
261726,999.0,"zinc oxide with icthammol, salicylic acid and ...",C0004057,"[T109, T121]",aspirin
261727,999.0,"zinc oxide with icthammol, salicylic acid and ...",C3547570,[T044],salicylic acid binding


In [184]:
drugs_umls_cleaned = drugs_umls[drugs_umls.TUI.str.join(" ").str.contains("T109") | 
           drugs_umls.TUI.str.join(" ").str.contains("T121") | 
           drugs_umls.TUI.str.join(" ").str.contains("T116") |
           drugs_umls.TUI.str.join(" ").str.contains("127") ][["drug_name", "CUI", "TUI", "name"]].reset_index(drop=True)

In [9]:
drugs_umls_cleaned = pd.read_feather(f"{path}/drug_names_umls_linked.feather")

In [10]:
""" UTS (UMLS Terminology Services) API client """
import json
from pathlib import Path
import requests
from lxml.html import fromstring


class UtsClient:
    """All the UTS REST API requests are handled through this client"""
    def __init__(self, apikey=None):
        if apikey is None:
            file_key = Path(__file__).resolve().parents[1] / 'uts-api-key.txt'
            self.apikey = open(file_key).read().rstrip()
            self.apikey = '47669881-3e07-4acb-b087-d9b767b02ce8'
        else:
            self.apikey = apikey
        self.service = "http://umlsks.nlm.nih.gov"
        self.headers = {
            "Content-type": "application/x-www-form-urlencoded",
            "Accept": "text/plain",
            "User-Agent": "python"
        }
        self.tgt = None
        self.base_uri = "https://uts-ws.nlm.nih.gov"
        self.version = "current"

    def gettgt(self):
        """Retrieve a ticket granting ticket"""
        auth_uri = "https://utslogin.nlm.nih.gov"
        params = {"apikey": self.apikey}
        auth_endpoint = "/cas/v1/api-key"
        r = requests.post(auth_uri + auth_endpoint, data=params,
                          headers=self.headers)
        response = fromstring(r.text)
        # extract the entire URL needed from the HTML form (action attribute)
        # returned - looks similar to
        # https://utslogin.nlm.nih.gov/cas/v1/tickets/TGT-36471-aYqNLN2rFIJPXKzxwdTNC5ZT7z3B3cTAKfSc5ndHQcUxeaDOLN-cas
        # we make a POST call to this URL in the getst method
        self.tgt = response.xpath('//form/@action')[0]

    def getst(self):
        """Request a single-use service ticket"""
        if self.tgt is None:
            self.gettgt()
        params = {"service": self.service}
        r = requests.post(self.tgt, data=params, headers=self.headers)
        return r.text

    def query_get(self, endpoint, query):
        r = requests.get(self.base_uri+endpoint, params=query)
        if r.status_code == 404:
            return
        return json.loads(r.text)
                   
    def get_concept_rxnorm_atoms(self, cui, tok):
        endpoint = f"/rest/content/{self.version}/CUI/{cui}/atoms"
        query = {"ticket": self.getst(), "sabs": "RXNORM"}
        rst = self.query_get(endpoint, query)
        # If MSH term can't be found by CUI, use the defining word to search
        if rst is None:
            endpoint = f"/rest/search/{self.version}"
            query = {"ticket": self.getst(),
                     "string": tok, "searchType": "exact"}
            rst = self.query_get(endpoint, query)
            if rst is not None:
                search_max = 5
                for rec in rst['result']['results']:
                    # search MSH until found
                    endpoint = (f"/rest/content/{self.version}"
                                f"/CUI/{rec['ui']}/atoms")
                    query = {"ticket": self.getst(), "sabs": "RXNORM"}
                    rst = self.query_get(endpoint, query)
                    if rst is not None:
                        return rst
                    else:
                        search_max -= 1
                    if search_max == 0:
                        return None
        return rst

In [12]:
umls_client = UtsClient(apikey="47669881-3e07-4acb-b087-d9b767b02ce8")

In [21]:
df_cui_to_rxnorm = drugs_umls_cleaned.copy()

def map_cui_to_rxnorm(i, cui, toks, umls_client):
    i_codes = []
    for tok in toks:
        req = umls_client.get_concept_rxnorm_atoms(cui=cui, tok=tok)
        try: code = req["result"][0]["code"].rsplit("/",1)[1]
        except: code = ""
        i_codes.append(code)
    return list(set(i_codes))
    
from joblib import Parallel, delayed
drug_rx_codes =  Parallel(n_jobs=40)(delayed(map_cui_to_rxnorm)(i, df_cui_to_rxnorm.iloc[i, 1], df_cui_to_rxnorm.iloc[i, 2], umls_client) for i in tqdm(range(len(df_cui_to_rxnorm))))


,drug_name,CUI,TUI,name,rx_code
0,Simvastatin 40mg tablets,C0074554,"[T109, T121]",simvastatin,[36567]
1,Bendroflumethiazide 2.5mg tablets,C0004975,"[T109, T121]",bendroflumethiazide,[1369]
2,Aspirin 75mg dispersible tablets,C0004057,"[T109, T121]",aspirin,[1191]
3,Omeprazole 20mg gastro-resistant capsules,C0028978,"[T109, T121]",omeprazole,[7646]
4,Omeprazole 20mg gastro-resistant capsules,C0771315,"[T109, T121]",omeprazole sodium,[1792108]
...,...,...,...,...,...
67619,"zinc oxide with icthammol, salicylic acid and ...",C0043491,"[T121, T197]",zinc oxide,[]
67620,"zinc oxide with icthammol, salicylic acid and ...",C0036079,"[T109, T121]",salicylic acid,[]
67621,"zinc oxide with icthammol, salicylic acid and ...",C0073983,"[T109, T121]",salsalate,[]
67622,"zinc oxide with icthammol, salicylic acid and ...",C0004057,"[T109, T121]",aspirin,[]


In [31]:
drug_rx_codes_cleaned = ["".join([x for x in drug if x]) for drug in drug_rx_codes]

In [32]:
df_cui_to_rxnorm["rx_code"] = drug_rx_codes_cleaned
df_cui_to_rxnorm

,drug_name,CUI,TUI,name,rx_code
0,Simvastatin 40mg tablets,C0074554,"[T109, T121]",simvastatin,36567
1,Bendroflumethiazide 2.5mg tablets,C0004975,"[T109, T121]",bendroflumethiazide,1369
2,Aspirin 75mg dispersible tablets,C0004057,"[T109, T121]",aspirin,1191
3,Omeprazole 20mg gastro-resistant capsules,C0028978,"[T109, T121]",omeprazole,7646
4,Omeprazole 20mg gastro-resistant capsules,C0771315,"[T109, T121]",omeprazole sodium,1792108
...,...,...,...,...,...
67619,"zinc oxide with icthammol, salicylic acid and ...",C0043491,"[T121, T197]",zinc oxide,
67620,"zinc oxide with icthammol, salicylic acid and ...",C0036079,"[T109, T121]",salicylic acid,
67621,"zinc oxide with icthammol, salicylic acid and ...",C0073983,"[T109, T121]",salsalate,
67622,"zinc oxide with icthammol, salicylic acid and ...",C0004057,"[T109, T121]",aspirin,


In [33]:
df_cui_to_rxnorm.to_feather(f"{path}/drug_names_umls_linked_rxnorm.feather")

In [14]:
df_cui_to_rxnorm

,drug_name,CUI,TUI,name,rx_code
0,Simvastatin 40mg tablets,C0074554,"[T109, T121]",simvastatin,[36567]
1,Bendroflumethiazide 2.5mg tablets,C0004975,"[T109, T121]",bendroflumethiazide,[1369]
2,Aspirin 75mg dispersible tablets,C0004057,"[T109, T121]",aspirin,[1191]
3,Omeprazole 20mg gastro-resistant capsules,C0028978,"[T109, T121]",omeprazole,[7646]
4,Omeprazole 20mg gastro-resistant capsules,C0771315,"[T109, T121]",omeprazole sodium,[1792108]
5,Levothyroxine sodium 100microgram tablets,C0079691,"[T116, T121, T125]",levothyroxine sodium,[40144]
6,Levothyroxine sodium 100microgram tablets,C0040165,"[T116, T121, T125]",levothyroxine,[10582]
7,Levothyroxine sodium 100microgram tablets,C1881373,"[T109, T121]",Synthetic Levothyroxine,[]
8,Amlodipine 5mg tablets,C0051696,"[T109, T121]",amlodipine,[17767]
9,Paracetamol 500mg tablets,C0000970,"[T109, T121]",acetaminophen,[161]
